In [2]:
# make sure ES is up and running
import requests
res = requests.get('http://localhost:9200')
print(res.content)

b'{\n  "name" : "KEW0000",\n  "cluster_name" : "elasticsearch",\n  "cluster_uuid" : "PePhqOwBS_-cg5VG6Iecew",\n  "version" : {\n    "number" : "6.2.1",\n    "build_hash" : "7299dc3",\n    "build_date" : "2018-02-07T19:34:26.990113Z",\n    "build_snapshot" : false,\n    "lucene_version" : "7.2.1",\n    "minimum_wire_compatibility_version" : "5.6.0",\n    "minimum_index_compatibility_version" : "5.0.0"\n  },\n  "tagline" : "You Know, for Search"\n}\n'


In [3]:

#connect to our cluster
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [4]:
#index some test data
es.index(index='test-index', doc_type='test', id=1, body={'test': 'test'})

{'_id': '1',
 '_index': 'test-index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'test',
 '_version': 1,
 'result': 'created'}

In [5]:

#delete test data and try with something more interesting
es.delete(index='test-index', doc_type='test', id=1)

{'_id': '1',
 '_index': 'test-index',
 '_primary_term': 1,
 '_seq_no': 1,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'test',
 '_version': 2,
 'result': 'deleted'}

In [6]:
es.index(index='sw', doc_type='people', id=1, body={
	"name": "Luke Skywalker",
	"height": "172",
	"mass": "77",
	"hair_color": "blond",
	"birth_year": "19BBY",
	"gender": "male",
})


{'_id': '1',
 '_index': 'sw',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'people',
 '_version': 1,
 'result': 'created'}

In [7]:
#let's iterate over swapi people documents and index them
import json
r = requests.get('http://localhost:9200') 
i = 1
while r.status_code == 200:
    r = requests.get('http://swapi.co/api/people/'+ str(i))
    es.index(index='sw', doc_type='people', id=i, body=json.loads(r.content))
    i=i+1
    
print(i)

18


In [9]:
es.get(index='sw', doc_type='people', id=5)

{'_id': '5',
 '_index': 'sw',
 '_source': {'birth_year': '19BBY',
  'created': '2014-12-10T15:20:09.791000Z',
  'edited': '2014-12-20T21:17:50.315000Z',
  'eye_color': 'brown',
  'films': ['https://swapi.co/api/films/2/',
   'https://swapi.co/api/films/6/',
   'https://swapi.co/api/films/3/',
   'https://swapi.co/api/films/1/',
   'https://swapi.co/api/films/7/'],
  'gender': 'female',
  'hair_color': 'brown',
  'height': '150',
  'homeworld': 'https://swapi.co/api/planets/2/',
  'mass': '49',
  'name': 'Leia Organa',
  'skin_color': 'light',
  'species': ['https://swapi.co/api/species/1/'],
  'starships': [],
  'url': 'https://swapi.co/api/people/5/',
  'vehicles': ['https://swapi.co/api/vehicles/30/']},
 '_type': 'people',
 '_version': 1,
 'found': True}